<a href="https://colab.research.google.com/github/Kat3473/star-tracker/blob/main/Polaris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Here is the following step-by-step workflow to train and implement the AI model onto the Arduino.


1.   Setup the Python environment on Colab
2.   Collect and upload the data
3.   Process the data through Colab
4.   Train & test the AI model on Colab
5.   Convert the model into TensorFlow Lite
6.   Encode the model as an Arduino header file and upload it onto the board



# Setup & Imports

Install the mediapipe library onto the Colab session. This will allow us to run object detection tasks directly on here. Perfrect for testing the model.

In [ ]:
!pip install mediapipe

In [22]:
!pip install tflite_support

Importing all neccessary libraries onto the drive:


*   Standard Python Libraries
*   TensorFlow and Keras
*   Libraries for image processing
*   Object Detection
*   Converting to TFLite & writing metadata




In [23]:
#Import: General
import numpy as np
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
import pathlib
from PIL import Image
import matplotlib.pyplot as plt

#Import: Image Processing
from keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import load_img, img_to_array

#Import: ML model
import tensorflow as tf
import keras

#Import: Object Detection
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

#Import: TFLite & Metadata writing
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils

# Dataset Construction

The images to train and test the model are grabbed directly from google drive to construct the correct datasets.

For this particular case, an image dataset of images of the night sky are being used. It is assumed that the brighest and only visible star in the night sky is Polaris.

In [3]:
#Defining dataset parameters
batch_size = 32 #Size of batches for the data
img_height = 224 #Height of the images
img_width = 224 #Width of the images

#Grabbing the images
dataset_url = "/content/drive/MyDrive/Star Tracker/Dataset" #google drive of the images
data_dir = pathlib.Path(dataset_url).with_suffix('') #sets up the directory to be used
classnames = ['polaris'] #Array to assign a label to the class

#Training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir, #the directory being converted
  validation_split=0.2, #20% used for validation, 80% for actual training
  subset="training", #this is the training subset being outputted
  seed=132, #seed to randomise data order and seperation
  image_size=(img_height, img_width), #converts image sizes to required
  batch_size=batch_size, #the batch size of the dataset
  class_names=classnames, #assinging labels to the classes
  label_mode='int' #controls the datatype of the class labels i.e. [1, 0]
  )

#Validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory( #same as above but for the validation dataset
  data_dir,
  validation_split=0.2,
  subset="validation", #for validation subset
  seed=132,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  class_names=classnames,
  label_mode='int'
  )


train_ds = train_ds.map(lambda images, labels:
                        (images, labels))
val_ds = val_ds.map(lambda images, labels:
                    (images, labels))


Found 61 files belonging to 1 classes.
Using 49 files for training.
Found 61 files belonging to 1 classes.
Using 12 files for validation.


# Model: Pre-trained

We'll be using transfer learning by building a much smaller 'model' onto the MobileNet neural network.

In [6]:
# Load the pre-trained model
model = keras.applications.MobileNetV3Small(
    input_shape=(img_height, img_width, 3),  # Define input shape
    alpha=1.0,
    minimalistic=False,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    classes=1000,
    pooling='avg',  # Add average pooling
    dropout_rate=0.2,
    classifier_activation="softmax",
    include_preprocessing=True,
    name="MobileNetV3Small",
)

num_classes = 1
x = model.output
x = keras.layers.Dense(num_classes, activation='relu')(x)
model = keras.models.Model(inputs=model.input, outputs=x)

# Training

In [7]:
#Compiling the model
model.compile(
  optimizer='adamax', #using adamax optimizer
  loss=tf.keras.losses.SparseCategoricalCrossentropy(), #using sparce categorical crossentropy loss function
  metrics=[keras.metrics.SparseCategoricalAccuracy() #using accuracy (multi-class model version) and poisson metrics
          ]
)

#Training the model
score = model.fit(
  train_ds, #training dataset
  validation_data=val_ds, #validation dataset
  epochs=1, #number of training steps i.e. number of times the model is trained using the training dataset
).history #Recording the metrics into the array 'score'

2/2 ━━━━━━━━━━━━━━━━━━━━ 104s 31s/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 1.0000


In [ ]:
#Plotting Loss per epoch
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,1])
line1 = plt.plot(score["loss"], label='Training')
line2 = plt.plot(score["val_loss"], label='Validation')
plt.legend()

#Plotting Accuracy per epoch
plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(score["sparse_categorical_accuracy"], label='Training')
plt.plot(score["val_sparse_categorical_accuracy"], label='Validation')
plt.legend()

print(score)

# Exporting

Here we will save, convert and export the trained model as a TFLite model file to be used. The exported model will be imported in the object detection section for testing and fine-tuning.

In [16]:
saved_model_dir = "/content/drive/MyDrive/Star Tracker/Model/Polaris/"
model.export(saved_model_dir + 'Polaris.h5')

Saved artifact at '/content/drive/MyDrive/Star Tracker/Model/Polaris/Polaris.h5'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134982157216528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157215568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157215952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157217488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157217104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157216720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157216912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982154510096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157217296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157216144: TensorSpec(shape=(), dt

In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Saved artifact at '/tmp/tmpqtzew144'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134982157216528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157215568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157215952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157217488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157217104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157216720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157216912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982154510096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157217296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134982157216144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13498215451240

# Writing Metadata

Once the model has been converted to TFLite, we need to write the metadata for it before it's useable for object detection testing.

In [25]:
_MODEL_PATH = "/content/drive/MyDrive/Star Tracker/Model/Polaris/Polaris.tflite"
_LABEL_FILE = "/content/drive/MyDrive/Star Tracker/Model/Polaris/labels.txt"
_SAVE_TO_PATH = "/content/drive/MyDrive/Star Tracker/Model/Polaris/Polaris_metadata.tflite"

In [26]:
ObjectDetectorWriter = object_detector.MetadataWriter

_INPUT_NORM_MEAN = 127.5
_INPUT_NORM_STD = 127.5

# Create the metadata writer.
writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [_INPUT_NORM_MEAN], [_INPUT_NORM_STD],
    [_LABEL_FILE])

# Verify the metadata generated by metadata writer.
print(writer.get_metadata_json())

# Populate the metadata into the model.
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

{
  "name": "ObjectDetector",
  "description": "Identify which of a known set of objects might be present and provide information about their positions within the given image or a video stream.",
  "subgraph_metadata": [
    {
      "input_tensor_metadata": [
        {
          "name": "image",
          "description": "Input image to be detected.",
          "content": {
            "content_properties_type": "ImageProperties",
            "content_properties": {
              "color_space": "RGB"
            }
          },
          "process_units": [
            {
              "options_type": "NormalizationOptions",
              "options": {
                "mean": [
                  127.5
                ],
                "std": [
                  127.5
                ]
              }
            }
          ],
          "stats": {
            "max": [
              1.0
            ],
            "min": [
              -1.0
            ]
          }
        }
      ],
     

# Object Detection

The visualize function will allow us to visualize the results from the object detection test scenarios.



In [10]:
MARGIN = 10  # pixels
ROW_SIZE = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
TEXT_COLOR = (255, 0, 0)  # red


def visualize(
    image,
    detection_result
) -> np.ndarray:
  """Draws bounding boxes on the input image and return it.
  Args:
    image: The input RGB image.
    detection_result: The list of all "Detection" entities to be visualize.
  Returns:
    Image with bounding boxes.
  """
  for detection in detection_result.detections:
    # Draw bounding_box
    bbox = detection.bounding_box
    start_point = bbox.origin_x, bbox.origin_y
    end_point = bbox.origin_x + bbox.width, bbox.origin_y + bbox.height
    cv2.rectangle(image, start_point, end_point, TEXT_COLOR, 3)

    # Draw label and score
    category = detection.categories[0]
    category_name = category.category_name
    probability = round(category.score, 2)
    result_text = category_name + ' (' + str(probability) + ')'
    text_location = (MARGIN + bbox.origin_x,
                     MARGIN + ROW_SIZE + bbox.origin_y)
    cv2.putText(image, result_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                FONT_SIZE, TEXT_COLOR, FONT_THICKNESS)

  return image

Here we will implement the object detection features.

Note: This is still in progress and is currently using an externally imported TFLite model for testing.

In [28]:
base_options = python.BaseOptions(model_asset_path='/content/drive/MyDrive/Star Tracker/Model/Polaris/Polaris_metadata.tflite')
options = vision.ObjectDetectorOptions(base_options=base_options,
                                       score_threshold=0.5)
detector = vision.ObjectDetector.create_from_options(options)

image = mp.Image.create_from_file('/content/drive/MyDrive/Star Tracker/RS1695_AL4I4805_ad78662bc5c0b4a1f0fa41c3e05dc51c.jpg')

detection_result = detector.detect(image)

image_copy = np.copy(image.numpy_view())
annotated_image = visualize(image_copy, detection_result)
rgb_annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
cv2_imshow(rgb_annotated_image)

ValueError: Expected a model with 2 or 4 output tensors, found 1.